<a href="https://colab.research.google.com/github/nisanuro/CNG562-Assignment-2/blob/master/CNG562_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics, datasets, preprocessing
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [0]:
def randomOneHoldout(X_train, Y_train):
  x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)
  return x_train, x_test, y_train, y_test

In [0]:
def stratifiedOneHoldout(X_train, Y_train):
  x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.3, random_state=0)
  return x_train, x_test, y_train, y_test

In [0]:
def kNN(k: int, metric: str, X_train, Y_train):
  
  #Model
  if metric == "mahalanobis" :
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', metric=metric, algorithm="brute", metric_params={'V': np.cov(X)})
  else:
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', metric=metric)

  #5-Fold
  cv_result_knn_5 = cross_val_score(knn, X_train, Y_train, cv=5, scoring='accuracy')
  
  #10-Fold
  cv_result_knn_10 = cross_val_score(knn, X_train, Y_train, cv=10, scoring='accuracy')
  
  #Random One Holdout
  x_train, x_test, y_train, y_test_random = randomOneHoldout(X_train, Y_train)
  knn.fit(x_train, y_train)
  y_pred_knn_random = knn.predict(x_test)
  
  #Stratified One Holdout
  x_train, x_test, y_train, y_test_stratified = stratifiedOneHoldout(X_train, Y_train)
  knn.fit(x_train, y_train)
  y_pred_knn_stratified = knn.predict(x_test)

  print("5 Fold")
  print("KNN Accuracy: ", cv_result_knn_5.mean())
  
  print("10 Fold")
  print("KNN Accuracy: ", cv_result_knn_10.mean())

  print("Random One Hold Out")
  print("KNN Accuracy: ", 1 - metrics.mean_squared_error(y_test_random, y_pred_knn_random))
  
  print("Stratified One Hold Out Fold")
  print("KNN Accuracy: ", 1 - metrics.mean_squared_error(y_test_stratified, y_pred_knn_stratified))

In [0]:
def displayAccuracy(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

    k = [3, 5, 7, 9, 11]
    metric = ["euclidean", "manhattan", "chebyshev", "mahalanobis", "minkowski", "wminkowski", "seuclidean"]

    for i in k:
      for j in metric:
          if metric != "wminkowski":
            kNN(i, j, X_train, Y_train)

In [25]:
if __name__ == '__main__':

  iris = datasets.load_iris()
  
  X = iris.data
  Y = iris.target
  
  # Z-Score
  scaler = StandardScaler()
  scaler.fit(X)
  z_score = scaler.transform(X)

  #Displaying result according to each type of methods and regression model
  print("\nRaw: ")
  displayAccuracy(X,Y)
  print("\nZ-Score: ")
  displayAccuracy(z_score,Y)


Raw: 
5 Fold
KNN Accuracy:  0.9428571428571428
10 Fold
KNN Accuracy:  0.9427272727272727
Random One Hold Out
KNN Accuracy:  0.96875
Stratified One Hold Out Fold
KNN Accuracy:  0.96875
5 Fold
KNN Accuracy:  0.9428571428571428
10 Fold
KNN Accuracy:  0.941818181818182
Random One Hold Out
KNN Accuracy:  0.9375
Stratified One Hold Out Fold
KNN Accuracy:  0.9375
5 Fold
KNN Accuracy:  0.9523809523809523
10 Fold
KNN Accuracy:  0.9518181818181818
Random One Hold Out
KNN Accuracy:  0.96875
Stratified One Hold Out Fold
KNN Accuracy:  0.96875
5 Fold
KNN Accuracy:  0.8285714285714285
10 Fold
KNN Accuracy:  0.8981818181818182
Random One Hold Out
KNN Accuracy:  0.9375
Stratified One Hold Out Fold
KNN Accuracy:  0.9375
5 Fold
KNN Accuracy:  0.9428571428571428
10 Fold
KNN Accuracy:  0.9427272727272727
Random One Hold Out
KNN Accuracy:  0.96875
Stratified One Hold Out Fold
KNN Accuracy:  0.96875


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: __init__() takes exactly 2 positional arguments (1 given)

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: __init__() takes exactly 2 positional arguments (1 given)

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: __init__() takes exactly 2 positional arguments (1 given)

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFaile

TypeError: ignored

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0, stratify=Y)

In [0]:
Train_x, TrainDev_x, Train_y, TrainDev_y = train_test_split(X_train, Y_train, test_size=0.2, random_state=0, stratify=Y_train)
 Dev_x, Test_x, Dev_y, Test_y = train_test_split(X_test, Y_test, test_size=0.5, random_state=0, stratify=Y_test)